# Shopkit

### Import Libraries

In [54]:
# import libraries 
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns

### Load Auth Tokens

In [42]:
# define function for getting auth tokens
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [43]:
# load auth token
auth_path = '/Users/akg/.secret/shopkit/api_keys.json'
auth_key = get_keys(auth_path)['auth-token']

### Define Endpoint

In [44]:
# define endpoint
endpoint_url = 'https://shopkit-caa72.uc.r.appspot.com/api/store/products/'

# define headers
headers = {
    "accept": "application/json",
    "Authorization": f"Token {auth_key}"
}

In [45]:
# brand-specific slugs
# halfdays
halfdays = 'halfdays.myshopify.com'

In [46]:
# create brand-specific endpoint 
halfday_endpoint = endpoint_url + halfdays

### Request Endpoint

In [47]:
# request endpoint
json_data = requests.get(url=halfday_endpoint, headers=headers).json()

### Examine Data

In [48]:
# preview data 
json_data['46 products'][0].keys()

dict_keys(['id', 'title', 'body_html', 'vendor', 'product_type', 'created_at', 'handle', 'updated_at', 'published_at', 'template_suffix', 'status', 'published_scope', 'tags', 'admin_graphql_api_id', 'variants', 'options', 'images', 'image'])

In [49]:
# save data as dataframe 
halfdays_df = pd.json_normalize(json_data['46 products'])

In [50]:
halfdays_df.columns

Index(['id', 'title', 'body_html', 'vendor', 'product_type', 'created_at',
       'handle', 'updated_at', 'published_at', 'template_suffix', 'status',
       'published_scope', 'tags', 'admin_graphql_api_id', 'variants',
       'options', 'images', 'image.id', 'image.position', 'image.created_at',
       'image.updated_at', 'image.alt', 'image.width', 'image.height',
       'image.src', 'image.variant_ids', 'image.admin_graphql_api_id',
       'image'],
      dtype='object')

In [51]:
# export to csv
halfdays_df.to_csv('/Users/akg/Documents/shopkit/clothing_recs/complete_product_lists/halfdays_unexploded.csv', index=False)

In [52]:
len(halfdays_df)

46

### Explode Options & Variants

In [30]:
# explode by options and variants
options_df = halfdays_df.explode('options')
variants_df = halfdays_df.explode('variants')

In [31]:
# pull data from variants
variants_df['variant_id'] = variants_df['variants'].apply(lambda x: x['id'])
variants_df['variant_title'] = variants_df['variants'].apply(lambda x: x['title'])
variants_df['price'] = variants_df['variants'].apply(lambda x: x['price'])
variants_df['sku'] = variants_df['variants'].apply(lambda x: x['sku'])
variants_df['position'] = variants_df['variants'].apply(lambda x: x['position'])
variants_df['inv_policy'] = variants_df['variants'].apply(lambda x: x['inventory_policy'])
variants_df['compare_at_price'] = variants_df['variants'].apply(lambda x: x['compare_at_price'])
variants_df['fulfillment_service'] = variants_df['variants'].apply(lambda x: x['fulfillment_service'])
variants_df['inv_management'] = variants_df['variants'].apply(lambda x: x['inventory_management'])
variants_df['option_1'] = variants_df['variants'].apply(lambda x: x['option1'])
variants_df['option_2'] = variants_df['variants'].apply(lambda x: x['option2'])
variants_df['option_3'] = variants_df['variants'].apply(lambda x: x['option3'])
variants_df['taxable'] = variants_df['variants'].apply(lambda x: x['taxable'])
variants_df['barcode'] = variants_df['variants'].apply(lambda x: x['barcode'])
variants_df['grams'] = variants_df['variants'].apply(lambda x: x['grams'])
variants_df['weight'] = variants_df['variants'].apply(lambda x: x['weight'])
variants_df['weight_unit'] = variants_df['variants'].apply(lambda x: x['weight_unit'])
variants_df['inventory_item_id'] = variants_df['variants'].apply(lambda x: x['inventory_item_id'])
variants_df['inventory_quantity'] = variants_df['variants'].apply(lambda x: x['inventory_quantity'])
variants_df['old_inventory_quantity'] = variants_df['variants'].apply(lambda x: x['old_inventory_quantity'])
variants_df['requires_shipping'] = variants_df['variants'].apply(lambda x: x['requires_shipping'])

In [32]:
# explode variants
halfdays_df = halfdays_df.explode('variants')

In [33]:
halfdays_df.head()

,id,title,body_html,vendor,product_type,created_at,handle,updated_at,published_at,template_suffix,...,image.position,image.created_at,image.updated_at,image.alt,image.width,image.height,image.src,image.variant_ids,image.admin_graphql_api_id,image
0,6044920250527,100% Mineral Sunscreen Stick,SKU 3133,Halfdays,,2020-12-31T11:21:43-07:00,100-mineral-sunscreen-stick,2022-10-25T11:57:47-06:00,None,,...,1.0,2020-12-31T11:21:46-07:00,2020-12-31T11:21:46-07:00,None,660.0,834.0,https://cdn.shopify.com/s/files/1/0427/2413/09...,[],gid://shopify/ProductImage/21032395178143,NaN
1,7847011975393,A Test Product,,Halfdays,,2022-09-23T01:52:16-06:00,test-product,2022-10-25T11:57:48-06:00,None,,...,1.0,2022-09-23T01:52:19-06:00,2022-10-06T02:05:06-06:00,Black,1806.0,2691.0,https://cdn.shopify.com/s/files/1/0427/2413/09...,"[43336439300321, 43336439365857, 4333643943139...",gid://shopify/ProductImage/38334953488609,NaN
1,7847011975393,A Test Product,,Halfdays,,2022-09-23T01:52:16-06:00,test-product,2022-10-25T11:57:48-06:00,None,,...,1.0,2022-09-23T01:52:19-06:00,2022-10-06T02:05:06-06:00,Black,1806.0,2691.0,https://cdn.shopify.com/s/files/1/0427/2413/09...,"[43336439300321, 43336439365857, 4333643943139...",gid://shopify/ProductImage/38334953488609,NaN
1,7847011975393,A Test Product,,Halfdays,,2022-09-23T01:52:16-06:00,test-product,2022-10-25T11:57:48-06:00,None,,...,1.0,2022-09-23T01:52:19-06:00,2022-10-06T02:05:06-06:00,Black,1806.0,2691.0,https://cdn.shopify.com/s/files/1/0427/2413/09...,"[43336439300321, 43336439365857, 4333643943139...",gid://shopify/ProductImage/38334953488609,NaN
1,7847011975393,A Test Product,,Halfdays,,2022-09-23T01:52:16-06:00,test-product,2022-10-25T11:57:48-06:00,None,,...,1.0,2022-09-23T01:52:19-06:00,2022-10-06T02:05:06-06:00,Black,1806.0,2691.0,https://cdn.shopify.com/s/files/1/0427/2413/09...,"[43336439300321, 43336439365857, 4333643943139...",gid://shopify/ProductImage/38334953488609,NaN


In [34]:
halfdays_df.iloc[0]['variants'].keys()

dict_keys(['id', 'title', 'price', 'sku', 'position', 'inventory_policy', 'compare_at_price', 'fulfillment_service', 'inventory_management', 'option1', 'option2', 'option3', 'created_at', 'updated_at', 'taxable', 'barcode', 'grams', 'image_id', 'weight', 'weight_unit', 'inventory_item_id', 'inventory_quantity', 'old_inventory_quantity', 'requires_shipping', 'admin_graphql_api_id'])

In [35]:
halfdays_df.iloc[4]['variants']['option1']

'Black'

In [36]:
halfdays_df.iloc[4]['variants']['option2']

'M'

In [37]:
halfdays_df.iloc[4]['variants']['option3']

'Short'

In [38]:
# pull out data from variants
halfdays_df['variant_id'] = halfdays_df['variants'].apply(lambda x: x['id'])
halfdays_df['variant_title'] = halfdays_df['variants'].apply(lambda x: x['title'])
halfdays_df['price'] = halfdays_df['variants'].apply(lambda x: x['price'])
halfdays_df['sku'] = halfdays_df['variants'].apply(lambda x: x['sku'])
halfdays_df['position'] = halfdays_df['variants'].apply(lambda x: x['position'])
halfdays_df['inv_policy'] = halfdays_df['variants'].apply(lambda x: x['inventory_policy'])
halfdays_df['compare_at_price'] = halfdays_df['variants'].apply(lambda x: x['compare_at_price'])
halfdays_df['fulfillment_service'] = halfdays_df['variants'].apply(lambda x: x['fulfillment_service'])
halfdays_df['inv_management'] = halfdays_df['variants'].apply(lambda x: x['inventory_management'])
halfdays_df['option_1'] = halfdays_df['variants'].apply(lambda x: x['option1'])
halfdays_df['option_2'] = halfdays_df['variants'].apply(lambda x: x['option2'])
halfdays_df['option_3'] = halfdays_df['variants'].apply(lambda x: x['option3'])
halfdays_df['taxable'] = halfdays_df['variants'].apply(lambda x: x['taxable'])
halfdays_df['barcode'] = halfdays_df['variants'].apply(lambda x: x['barcode'])
halfdays_df['grams'] = halfdays_df['variants'].apply(lambda x: x['grams'])
halfdays_df['weight'] = halfdays_df['variants'].apply(lambda x: x['weight'])
halfdays_df['weight_unit'] = halfdays_df['variants'].apply(lambda x: x['weight_unit'])
halfdays_df['inventory_item_id'] = halfdays_df['variants'].apply(lambda x: x['inventory_item_id'])
halfdays_df['inventory_quantity'] = halfdays_df['variants'].apply(lambda x: x['inventory_quantity'])
halfdays_df['old_inventory_quantity'] = halfdays_df['variants'].apply(lambda x: x['old_inventory_quantity'])
halfdays_df['requires_shipping'] = halfdays_df['variants'].apply(lambda x: x['requires_shipping'])

## Save Dataframe as a CSV

In [40]:
# output to csv
halfdays_df.to_csv('complete_product_lists/halfdays_products.csv', index=False)